In [23]:
import jax
from jax import numpy as jnp
import numpy as np

def f(a,b):
    c = a**2
    d = b+a
    return c*d

In [18]:
jax.make_jaxpr(f)(2,3)

{ lambda ; a:i32[] b:i32[]. let
    c:i32[] = integer_pow[y=2] a
    d:i32[] = add b a
    e:i32[] = mul c d
  in (e,) }

In [20]:
vec_f = jax.vmap(f)
jax.make_jaxpr(vec_f)(jnp.array([1,2,3]),jnp.array([4,5,6]))

{ lambda ; a:i32[3] b:i32[3]. let
    c:i32[3] = integer_pow[y=2] a
    d:i32[3] = add b a
    e:i32[3] = mul c d
  in (e,) }

In [21]:
vec_f = jax.vmap(f,[0,None])
jax.make_jaxpr(vec_f)(jnp.array([1,2,3]),4)

{ lambda ; a:i32[3] b:i32[]. let
    c:i32[3] = integer_pow[y=2] a
    d:i32[] = convert_element_type[new_dtype=int32 weak_type=False] b
    e:i32[3] = add d a
    f:i32[3] = mul c e
  in (f,) }

In [22]:
vec_f = jax.vmap(f,[None,0])
jax.make_jaxpr(vec_f)(1,jnp.array([2,3,4]))

{ lambda ; a:i32[] b:i32[3]. let
    c:i32[] = integer_pow[y=2] a
    d:i32[] = convert_element_type[new_dtype=int32 weak_type=False] a
    e:i32[3] = add b d
    f:i32[] = convert_element_type[new_dtype=int32 weak_type=False] c
    g:i32[3] = mul f e
  in (g,) }

In [28]:
def f(a,b):
    c = a @ b
    return c

jax.make_jaxpr(f)(np.random.randn(2,3),np.random.randn(3,4))

{ lambda ; a:f32[2,3] b:f32[3,4]. let
    c:f32[2,4] = dot_general[
      dimension_numbers=(([1], [0]), ([], []))
      preferred_element_type=float32
    ] a b
  in (c,) }

In [29]:
jax.make_jaxpr(jax.vmap(f))(np.random.randn(10,2,3),np.random.randn(10,3,4))

{ lambda ; a:f32[10,2,3] b:f32[10,3,4]. let
    c:f32[10,2,4] = dot_general[
      dimension_numbers=(([2], [1]), ([0], [0]))
      preferred_element_type=float32
    ] a b
  in (c,) }

In [47]:
def f(a,b):
    c = jnp.sum(a,axis=0) @ jnp.sum(b,axis=0)
    return c

jax.make_jaxpr(f)(np.random.randn(5,2,3),np.random.randn(6,3,4))

{ lambda ; a:f32[5,2,3] b:f32[6,3,4]. let
    c:f32[2,3] = reduce_sum[axes=(0,)] a
    d:f32[3,4] = reduce_sum[axes=(0,)] b
    e:f32[2,4] = dot_general[
      dimension_numbers=(([1], [0]), ([], []))
      preferred_element_type=float32
    ] c d
  in (e,) }

In [46]:
jax.make_jaxpr(jax.vmap(f,[0,1]))(np.random.randn(10,5,2,3),np.random.randn(6,10,3,4))

TypeError: f() takes 1 positional argument but 2 were given

In [39]:
def f(x):
    def g(carry, x):
        return carry+x, carry+x
    return jax.lax.scan(g,0,x)

jax.make_jaxpr(f)(np.random.randn(10))

{ lambda ; a:f32[10]. let
    b:f32[] c:f32[10] = scan[
      jaxpr={ lambda ; d:f32[] e:f32[]. let
          f:f32[] = add d e
          g:f32[] = add d e
        in (f, g) }
      length=10
      linear=(False, False)
      num_carry=1
      num_consts=0
      reverse=False
      unroll=1
    ] 0.0 a
  in (b, c) }

In [41]:
jax.make_jaxpr(jax.vmap(f))(np.random.randn(5,10))

{ lambda ; a:f32[5,10]. let
    b:f32[5] = broadcast_in_dim[broadcast_dimensions=() shape=(5,)] 0.0
    c:f32[10,5] = transpose[permutation=(1, 0)] a
    d:f32[5] e:f32[10,5] = scan[
      jaxpr={ lambda ; f:f32[5] g:f32[5]. let
          h:f32[5] = add f g
          i:f32[5] = add f g
        in (h, i) }
      length=10
      linear=(False, False)
      num_carry=1
      num_consts=0
      reverse=False
      unroll=1
    ] b c
    j:f32[5,10] = transpose[permutation=(1, 0)] e
  in (d, j) }

In [45]:
jax.vmap(jnp.sin,None,axis_size=5)(2.0)

Array([0.9092974, 0.9092974, 0.9092974, 0.9092974, 0.9092974],      dtype=float32, weak_type=True)

In [44]:
jax.vmap?